# Background Plotting Notebook

## Import python libraries

In [ ]:
import numpy as np 
import h5py
import matplotlib.pyplot as plt
import scipy.io
import os
import re
from utils import *

## Next define keys (data parameters) to be loaded and define function to load data
#### (1) keys_to_combine: some keys loaded for each shot & stored per shot (2) keys_to_sum: some keys loaded per each run and added (3) keys_to_check : check if some keys exits and have same values in all runs and load these keys 

# Load the data into defined variables
#### note if there is some error loading a key, then comment it out and try to load data. 
#### sometimes keys of a data parameter may be saved with different name, print out all keys to figure out the required key

In [ ]:
###############################################
runNumbers = []  # enter the run numbers to be loaded
folder = '/sdf/data/lcls/ds/cxi/cxil1037623/hdf5/smalldata/' # insert here the directory path where .h5 files are stored
###############################################
# (1) keys_to_combine: some keys loaded for each shot & stored per shot 
# (2) keys_to_sum: some keys loaded per each run and added 
# (3) keys_to_check : check if some keys exits and have same values in all runs and load these keys 
keys_to_combine = ['jungfrau4M/azav_mask0_azav', # Unfiltered
                   'jungfrau4M/azav_mask1_azav', # Filtered
                   'ipm_dg2/sum',
                    #'ipm_dg3/sum',
                   #'ipm_hfx_dg2/sum',
                   'gas_detector/f_11_ENRC',
                   'ebeam/photon_energy',
                   'evr/code_183',
                   'evr/code_137',
                   'evr/code_141',
                   'lightStatus/xray',
                  #'CXI-DG2-BMMON-WF/cxi_dg2_bmmon_wf_rebin_data',
                  'jungfrau4M/Full_thres_sum',
                  'feeBld/hproj']

keys_to_sum = ['Sums/jungfrau4M_calib']
#               'Sums/jungfrau4M_calib_thresADU1']

keys_to_check = ['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_q',
                'UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_q',
                'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_qbin',
                'UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_qbin',
                'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_qbins',
                'UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_qbins',
                'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_userMask',
                'UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_userMask',
                'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_matrix_q', # This are only needed once
                'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_matrix_phi', # This are only needed once
                'UserDataCfg/jungfrau4M/x',
                'UserDataCfg/jungfrau4M/y',
                'UserDataCfg/jungfrau4M/z',
                'UserDataCfg/jungfrau4M/cmask']
# Load the data in
data = combineRuns(runNumbers, folder, keys_to_combine, keys_to_sum, keys_to_check, verbose=False)  # this is the function to load the data with defined keys

# Unfiltered Data
azav = np.squeeze(data['jungfrau4M/azav_mask0_azav']) # I(q) : 1D azimuthal average of signals in each q bin
qbin = data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_qbin'] # q bin-size
q = data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_q'] # q bins 
qbins = data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_qbins'] # q bins
userMask = data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_userMask'].astype(bool) # User mask for this region
# Filtered Data
azavFiltered = np.squeeze(data['jungfrau4M/azav_mask1_azav']) # I(q) : 1D azimuthal average of signals in each q bin
qbinFiltered = data['UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_qbin'] # q bin-size
qFiltered = data['UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_q'] # q bins 
qbinsFiltered = data['UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_qbins'] # q bins
userMaskFiltered = data['UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_userMask'].astype(bool) # User mask for this region
# Other Data
matrix_q = data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_matrix_q'].reshape(8,512,1024) # Q values J4M shaped
matrix_phi = data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_matrix_phi'].reshape(8,512,1024) # phi valyes J4M shaped
xrayOn = data['evr/code_137'].astype(bool)  # xray on events
xrayOn2 = data['lightStatus/xray'].astype(bool)  # xray on events
jungfrau_sum = data['Sums/jungfrau4M_calib']  # Total Jungfrau detector counts summed in a run
#jungfrau_sum = data['Sums/jungfrau4M_calib_thresADU1']   # Total Jungfrau detector counts with Thresholds added, summed in a run 
x = data['UserDataCfg/jungfrau4M/x'] # coordinates of Jungfrau detector x,y,z
y = data['UserDataCfg/jungfrau4M/y']
z = data['UserDataCfg/jungfrau4M/z'] 

cmask = data['UserDataCfg/jungfrau4M/cmask'].astype(bool) # Mask for detector created 
run_indicator = data['run_indicator'] # run indicator for each shot
# dg3 = data['ipm_dg3/sum']    # downstream diode x-ray intensity
# pressure = data['epicsAll/gasCell_pressure']  # pressure in gas cell
xray_energy = data['gas_detector/f_11_ENRC']   # xray energy from gas detector (not calibrated to actual values)
xray_eV = data['ebeam/photon_energy']    # x-ray energy energy in eV
#dg2Trace = data['CXI-DG2-BMMON-WF/cxi_dg2_bmmon_wf_rebin_data']
numPhotons = data['jungfrau4M/Full_thres_sum']
spec = data['feeBld/hproj'] # Shot to shot spectrometer

# plot the raw Jungfrau Sum as a check

In [ ]:
#jungfrau_sum[jungfrau_sum>100*np.median(jungfrau_sum)]=0 
plt.figure(figsize=(9,6))
pcm = plot_jungfrau(y,-x,jungfrau_sum,vmin=0,vmax=1e4)
plt.colorbar(pcm)
plt.show()

# Runs compare : background signal


In [ ]:
runtype = ['Background'] # Edit this as needed to label runs before saving the data
azav_run = np.zeros(((azav[0,:].size), np.unique(run_indicator).size)) # I(q) per run
j = 0 
for run in runNumbers: # Looping through the runs, 
    timebin_run = np.where((run_indicator == run))[0] # Making a mask for separating out the runs from the combined dataset
    print(timebin_run.size) # Print the number of shots in the run
    azav_run[:,j] = np.nanmean(azav[timebin_run],axis=0) # Averaging the shot to shot azav data
    plt.plot(qbins[:-1], np.log10(azav_run[:, j]), label=f'Run {run} {runtype[j]}') # Plotting it
    j +=1

# Making the plot look nice
plt.xlabel('q')
plt.ylabel('log[I]')
plt.legend(loc=(1.1,0.2))
plt.title('Background Runs')
plt.show()

## Save the data to the backgrounds folder if you want
### This is generally useful if the dimensions of the arrays are different run to run.

In [ ]:
##########################
runType = 'Background' # Edit this for the type of run
backgroundFolder = 'Backgrounds' # Edit this if you want
##########################
bkgAzav = (azav_run[:,0]) # Taking only the first element of the array for saving
# np.savez(f'{backgroundFolder}/Background_Run_{runNumbers[0]}.npz',x=qbins[:-1],y=np.log10(y))
if not os.path.isdir(backgroundFolder): # Checking to see if the folder exists
    os.makedirs(backgroundFolder) # Making it if it doesn't
np.savez(f'{backgroundFolder}/Run_{runNumbers[0]}_{runType}.npz',x=qbins[:-1],y=np.log10(bkgAzav))

## Plot background for each run from the data in the background folder

In [ ]:
import scipy.io
plt.figure()
# Regular expression pattern for matching filenames
pattern = re.compile(r'^Run_(\d+)_(.+?)\.npz$')

# Loop through all files in the background folder
for filename in os.listdir(backgroundFolder):
    match = pattern.match(filename) # Check to see if the filename matches the pattern
    if match:
        parts = filename.split('_')
        # Extract the run number (second part)
        run_number = int(parts[1])
        # Extract the title (third part, remove .npz)
        title = parts[2].replace('.npz', '')
        # Load the .npz file
        file_path = os.path.join(backgroundFolder, filename)
        data = np.load(file_path)
        # Plot it
        plt.plot(data['x'],data['y'],linestyle='-',marker = 'o',linewidth = 1.5,label=f'Run {run_number} {title}')
    else:
        print(f'Skipped File: {filename}')

# Making the plot look nice
plt.xlabel('q')
plt.ylabel('log[I]')
plt.legend(loc=(1.1,0.2))
#plt.xlim(0,16)
#plt.ylim(-0.4,0.8)
plt.title('Background Runs')
plt.show()